LangChain 프롬프트 템플릿
```
LLM 프롬프트를 동적으로 구성하고 재 사용할 수 있도록 해 주는 도구
    단일입력 : 하나의 변수로 구성된 프롬프트 템플릿
    다중입력 : 둘 이상의 변수를 사용하는 템플릿
    ChatPromptTemplate 역할 기반 프롬프트 : 시스템/사용자 역활별 프롬프트 구성 .from_message()
    PartialPromptTemplate 활용 : 프롬프트 일부를 미리 고정하고 부분포멧팅을 사용(ex 시스템 메시지는 고정,,)
    프롬프트 출력 및 체인 실행 : LCEL
    프롬프트 작성 팁: 주의사항 및 모범사례
```

In [1]:
#  환경설정
%pip install --quiet langchain langchain-openai python-dotenv 

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# 단일 프롬프트 사용
from langchain.prompts import PromptTemplate
# 템플릿 문자열 정의
template_str = (
    "당신은 최고 수준의 마케팅 카피라이터 입니다.\n"
    "아래 제품의 매력적인 홍보문구를 100자 내외로 작성해 주세요.\n\n"
    "제품명 : {product_name}"
)
# PromptTemplate 객체 생성
product_prompt = PromptTemplate.from_template(template_str)
# 탬플릿에 값 채우기
formatted_prompt = product_prompt.format(product_name="이클립스")
print(formatted_prompt)

당신은 최고 수준의 마케팅 카피라이터 입니다.
아래 제품의 매력적인 홍보문구를 100자 내외로 작성해 주세요.

제품명 : 이클립스


In [4]:
# 프롬프트 | llm  -> invoke
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# Runnable 객체 생성  LCEL
chain = product_prompt | llm
result = chain.invoke({"product_name": "커피"})
print(result.content)

"하루의 시작을 특별하게! 깊고 풍부한 향의 커피 한 잔으로 당신의 순간을 깨워보세요. 진정한 여유를 느껴보세요!"


In [5]:
# 다중 입력
# 다중입력 템플릿 문자열 정의
multi_template_str = (
    "아래는 뉴스 기사 제목과 키워드 입니다.\n"
    "이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하세요\n\n"
    "제목:{title}\n"
    "키워드:{keyword}"
)
# 프롬프트 탬플릿 작성
summary_prompt = PromptTemplate(template=multi_template_str, input_variables=["title","keyword"])
# 포멧팅을 통해 프롬프트 값 확인
sample_title = '인공지능 기술의 발전과 미래'
sample_keyword = '머신러닝,딥러닝,LLM,랭체인,산업 혁신'
formatted_summary_promptt = summary_prompt.format(title = sample_title, keyword=sample_keyword)
print(formatted_summary_promptt)

아래는 뉴스 기사 제목과 키워드 입니다.
이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하세요

제목:인공지능 기술의 발전과 미래
키워드:머신러닝,딥러닝,LLM,랭체인,산업 혁신


In [6]:
# LCEL 출력
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
result_summary = (summary_prompt | llm | parser).invoke({
    'title':  sample_title,
    'keyword':sample_keyword
})
result_summary

'인공지능 기술의 발전은 머신러닝과 딥러닝의 혁신을 통해 가속화되고 있으며, 특히 대규모 언어 모델(LLM)과 랭체인 기술이 주목받고 있습니다. 이러한 기술들은 다양한 산업 분야에서 혁신을 이끌어내고 있으며, 기업들은 이를 활용하여 효율성을 높이고 새로운 비즈니스 모델을 창출하고 있습니다. 앞으로 인공지능은 더욱 발전하여 우리의 삶과 산업 구조에 큰 변화를 가져올 것으로 기대됩니다.'

In [7]:
# ChatPromptTemplate과 역활 기반 프롬프트
# 시스템/사용자/어시스턴트 역활(role)
# 시스템 메시지: 모델의 동작을 지시
# 사용자 메시지: 실제 사용자의 입력
# 어시스턴트 메시지: 이전 모델이 응답한 내용이 있드면 대화맥락유지를 위해 활용용

In [8]:
from langchain.prompts import ChatPromptTemplate
system_message = ("당신은 python 분야의 뛰어난 전문가이자 조언자 입니다."
                  "사용자의 프로그래밍 질문에 대해 친절하고 이해하기 쉽게 답변해 주세요")
user_message = "{question}"
chat_prompt = ChatPromptTemplate.from_messages([
    ("system",system_message),
    ("user",user_message)
])
# 템플릿을 이용해서 문장을 완성해 보세요 
sample_question = '파이썬의 클래스에 대해서 설명해주세요'
formated_message = chat_prompt.format_messages(question=sample_question)
print(formated_message)
# 파이프라인을 이용해서 llm호출 및 출력
answer = (chat_prompt | llm | parser).invoke({'question' : sample_question})
print(answer)

[SystemMessage(content='당신은 python 분야의 뛰어난 전문가이자 조언자 입니다.사용자의 프로그래밍 질문에 대해 친절하고 이해하기 쉽게 답변해 주세요', additional_kwargs={}, response_metadata={}), HumanMessage(content='파이썬의 클래스에 대해서 설명해주세요', additional_kwargs={}, response_metadata={})]
파이썬의 클래스는 객체 지향 프로그래밍(OOP)의 기본 개념 중 하나로, 객체를 생성하기 위한 청print(blueprint)입니다. 클래스는 속성(데이터)과 메서드(함수)를 포함할 수 있으며, 이를 통해 관련된 데이터와 기능을 하나의 단위로 묶을 수 있습니다.

### 클래스의 기본 구조

클래스를 정의하는 기본적인 문법은 다음과 같습니다:

```python
class ClassName:
    def __init__(self, parameter1, parameter2):
        self.attribute1 = parameter1
        self.attribute2 = parameter2

    def method_name(self):
        # 메서드의 내용
        pass
```

### 주요 구성 요소

1. **클래스 정의**: `class ClassName:` 구문을 사용하여 클래스를 정의합니다. `ClassName`은 클래스의 이름으로, 일반적으로 대문자로 시작합니다.

2. **생성자 (`__init__` 메서드)**: 클래스의 인스턴스가 생성될 때 호출되는 특별한 메서드입니다. 이 메서드에서 객체의 초기 속성을 설정할 수 있습니다. `self`는 현재 객체를 참조하는 매개변수입니다.

3. **속성**: 클래스 내에서 정의된 변수로, 객체의 상태를 나타냅니다. `self.attribute` 형태로 정의합니다.

4. **메서드**: 클래스 내에서 정의된 함수로, 객체가 수행할 수 있는 동작을 정의합니다.

In [9]:
# ParitalPromptTemplate : 탬플릿의 일부를 부분적으로 채운 새로운 탬플릿릿
from langchain.prompts import SystemMessagePromptTemplate,HumanMessagePromptTemplate
role_system_template = '당신은 {role} 분야의 전문 지식인 입니다. 가능한 자세히 답변해 주세요'
system_prompt = SystemMessagePromptTemplate.from_template(role_system_template)
user_prompt = HumanMessagePromptTemplate.from_template("{question}")

# chatprompttemplate을 생성
base_chat_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

partial_chat_prompt =  base_chat_prompt.partial(role = '주식 투자')

In [10]:
# partial로 생성된 프롬프트에 질문만 채워 프롬프트 구성
sample_question = "현재 2025년 5월 시장 시장 상황에서 삼성전자 주식 전망은"
message = partial_chat_prompt.format_messages(question = sample_question)
print(message)

[SystemMessage(content='당신은 주식 투자 분야의 전문 지식인 입니다. 가능한 자세히 답변해 주세요', additional_kwargs={}, response_metadata={}), HumanMessage(content='현재 2025년 5월 시장 시장 상황에서 삼성전자 주식 전망은', additional_kwargs={}, response_metadata={})]


In [12]:
# LCEL - invoke
answer = (partial_chat_prompt | llm | parser).invoke({'question':sample_question})
print(answer)

2025년 5월의 삼성전자 주식 전망을 평가하기 위해서는 여러 가지 요소를 고려해야 합니다. 다음은 삼성전자의 주식 전망에 영향을 미칠 수 있는 주요 요인들입니다.

### 1. **글로벌 반도체 시장 동향**
삼성전자는 반도체 산업에서 중요한 위치를 차지하고 있습니다. 2025년에는 인공지능(AI), 5G, IoT(사물인터넷) 등 다양한 기술의 발전으로 반도체 수요가 증가할 것으로 예상됩니다. 특히 메모리 반도체 시장의 회복 여부가 삼성전자의 실적에 큰 영향을 미칠 것입니다. 만약 반도체 가격이 안정세를 찾고 수요가 증가한다면, 삼성전자의 실적 개선이 기대됩니다.

### 2. **스마트폰 및 전자제품 시장**
삼성전자는 스마트폰, 가전제품 등 다양한 전자제품을 제조하고 있습니다. 2025년에는 새로운 기술이 적용된 스마트폰 모델이 출시될 가능성이 높으며, 이러한 혁신이 소비자에게 긍정적인 반응을 얻는다면 매출 증가에 기여할 수 있습니다. 또한, 지속적인 프리미엄 제품 라인업 강화가 중요합니다.

### 3. **글로벌 경제 상황**
2025년의 글로벌 경제 상황도 삼성전자의 주식에 큰 영향을 미칠 것입니다. 경제 성장률, 금리, 인플레이션 등 다양한 경제 지표가 삼성전자의 수익성에 영향을 미칠 수 있습니다. 특히, 주요 시장인 미국, 중국, 유럽의 경제 상황이 중요합니다.

### 4. **경쟁 상황**
삼성전자는 애플, 화웨이, TSMC 등과 같은 강력한 경쟁자들과 경쟁하고 있습니다. 경쟁사들의 기술 발전 및 시장 점유율 변화가 삼성전자의 시장 위치에 영향을 미칠 수 있습니다. 특히, 중국 기업들의 성장세가 삼성전자의 시장 점유율에 미치는 영향도 주목해야 합니다.

### 5. **정치적 및 규제 환경**
한국과 주요 국가 간의 무역 관계, 반도체 관련 규제, 환경 규제 등도 삼성전자의 운영에 영향을 미칠 수 있습니다. 특히, 미국과 중국 간의 무역 갈등이 지속된다면 삼성전자의 글로벌 공급망에 영향을 줄 수 있습니다.

### 6. **ESG(환경, 사회